In [41]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [42]:
# tfds works in both Eager and Graph modes
tf.executing_eagerly()

True

In [43]:
# Load the full GMD with MIDI only (no audio) as a tf.data.Dataset
dataset = tfds.load(
    name="groove/full-midionly",
    split=tfds.Split.TRAIN,
    try_gcs=True)

In [44]:
# Build your input pipeline
dataset = dataset.shuffle(1024).batch(64).prefetch(
    tf.data.experimental.AUTOTUNE)

In [45]:

for features in dataset.take(1):
  # Access the features you are interested in
  midi, genre = features["midi"], features["style"]["primary"]

In [40]:
midi[0]

<tf.Tensor: shape=(), dtype=string, numpy=b'MThd\x00\x00\x00\x06\x00\x00\x00\x01\x01\xe0MTrk\x00\x00\x04\x18\x00\xff\x03\nMidi Drums\x00\xff\x04\x08Brooklyn\x00\xffX\x04\x04\x02\x18\x08\x00\xffY\x02\x00\x00\x00\xffT\x05!\x00\x00\x00\x00\x00\xffQ\x03\t\'\xc0\x17\x99$n\t\xb9\x04Z\x00\x99*\x7fH\x89$@\t*@\x81\x15\xb9\x04Z\x00\x99*$Q\x89*@\x81\x14\x99$|\x05\xb9\x04Z\x00\x99*\x7fL\x89$@\x05*@\x81\x16\xb9\x04Z\x00\x99*\x13Q\x89*@\x81\x10\x99$J\x10\xb9\x04Z\x00\x99*\x7fA\x89$@\x10*@\x81\x10\xb9\x04Z\x00\x99*.Q\x89*@\x81\x05\x99$k\x10\xb9\x04Z\x00\x99*\x7f\x04&\x7f=\x89$@\x11*@\x03&@\x81\x1f\xb9\x04Z\x00\x99*\x12R\x89*@\x81\x05\x99$h\x08\xb9\x04Z\x00\x99*rI\x89$@\t*@\x81\x1c\xb9\x04Z\x00\x99*#Q\x89*@\x81\x17\x99$o\x0e\xb9\x04Z\x00\x99*\x7fC\x89$@\x0e*@\x81\x1f\xb9\x04Z\x00\x99*0Q\x89*@\x81\x1d\x99$a\r\xb9\x04Z\x00\x99*\x7fD\x89$@\r*@\x81\x0b\xb9\x04Z\x00\x99*0Q\x89*@\x81\x03\x99$\x7f\x0f\xb9\x04Z\x00\x99*\x7f\x02&\x7f?\x89$@\x11&@\x00*@\x81@\xb9\x04Z\x00\x99*5Q\x89*@\x81\x18\x99$c\x0c\xb9\x04Z\